In [ ]:
import os
os.environ["PYTENSOR_FLAGS"] = "cxx="

# ============================================================
# CONFIGURACION GLOBAL — Modelo 4: Regresion de Poisson Bayesiana
# ============================================================
RANDOM_SEED = 42
TEST_SIZE = 0.20

# MCMC (NUTS)
TUNE_SAMPLES = 1000
DRAW_SAMPLES = 1000
TARGET_ACCEPT = 0.90

# Umbral overfitting (diferencia relativa MAE train-val)
OVERFIT_THRESHOLD = 0.05

# 04 — Modelo 4: Regresion de Poisson Bayesiana

## Objetivo

Modelar **quantity_sold** (conteo de unidades vendidas) usando una distribucion de **Poisson Bayesiana**.

### Predictores

- `discount_percent`
- `rating`
- `is_weekend`

> Pipeline: EDA rapido, preparacion, modelo PyMC, diagnostico, metricas, overfitting, residuos, efectos y persistencia.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import joblib
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.rcParams.update({
    "figure.dpi": 110,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "font.size": 11,
})
PALETTE = {"train": "#2196F3", "val": "#FF5722", "fit": "#4CAF50"}

print("✅ Librerias importadas")
print(f"PyMC: {pm.__version__} | ArviZ: {az.__version__}")

## 1 — Carga y EDA rapido

In [ ]:
DATA_PATH = Path("../data/raw/amazon_sales_dataset.csv")
df = pl.read_csv(DATA_PATH)

print(f"Shape: {df.shape}")
print("Columnas:", df.columns)
df.head(4)

In [ ]:
# Preparacion de features
df_pd = df.to_pandas()
df_pd["order_date"] = pd.to_datetime(df_pd["order_date"])
df_pd["is_weekend"] = df_pd["order_date"].dt.dayofweek.isin([5, 6]).astype(int)

FEATURES = ["discount_percent", "rating", "is_weekend"]
TARGET = "quantity_sold"

X = df_pd[FEATURES].copy()
y = df_pd[TARGET].copy()

y_bins = pd.qcut(y, q=5, labels=False, duplicates="drop")
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y_bins
)

print(f"Train: {len(X_train):,} | Val: {len(X_val):,}")
print(f"Train mean/var: {y_train.mean():.2f}/{y_train.var():.2f}")
print(f"Val mean/var: {y_val.mean():.2f}/{y_val.var():.2f}")

## 2 — Estandarizacion y modelo PyMC

$$\log(\lambda_i) = \alpha + \beta_d \cdot discount^* + \beta_r \cdot rating^* + \beta_w \cdot is\_weekend$$

$$y_i \sim \text{Poisson}(\lambda_i)$$

In [ ]:
discount_mean = X_train["discount_percent"].mean()
discount_std = X_train["discount_percent"].std()
rating_mean = X_train["rating"].mean()
rating_std = X_train["rating"].std()

def standardize(X_df, d_mean, d_std, r_mean, r_std):
    Xs = X_df.copy()
    Xs["discount_scaled"] = (Xs["discount_percent"] - d_mean) / d_std
    Xs["rating_scaled"] = (Xs["rating"] - r_mean) / r_std
    return Xs

X_train_s = standardize(X_train, discount_mean, discount_std, rating_mean, rating_std)
X_val_s = standardize(X_val, discount_mean, discount_std, rating_mean, rating_std)

d_train = X_train_s["discount_scaled"].values.astype(float)
r_train = X_train_s["rating_scaled"].values.astype(float)
w_train = X_train_s["is_weekend"].values.astype(float)
y_train_np = y_train.values.astype(int)

d_val = X_val_s["discount_scaled"].values.astype(float)
r_val = X_val_s["rating_scaled"].values.astype(float)
w_val = X_val_s["is_weekend"].values.astype(float)
y_val_np = y_val.values.astype(int)

with pm.Model() as poisson_model:
    alpha = pm.Normal("alpha", mu=np.log(y_train_np.mean()), sigma=1.0)
    beta_d = pm.Normal("beta_d", mu=0, sigma=0.5)
    beta_r = pm.Normal("beta_r", mu=0, sigma=0.5)
    beta_w = pm.Normal("beta_w", mu=0, sigma=0.3)

    log_lambda = alpha + beta_d * d_train + beta_r * r_train + beta_w * w_train
    pm.Poisson("y_obs", mu=pm.math.exp(log_lambda), observed=y_train_np)

    trace = pm.sample(
        draws=DRAW_SAMPLES,
        tune=TUNE_SAMPLES,
        target_accept=TARGET_ACCEPT,
        random_seed=RANDOM_SEED,
        progressbar=True,
        return_inferencedata=True,
    )

print("✅ Muestreo completado")

## 3 — Diagnostico MCMC
Verificar `r_hat <= 1.01` y `ess_bulk >= 400`.

In [ ]:
summary = az.summary(trace, var_names=["alpha", "beta_d", "beta_r", "beta_w"], round_to=4)
display(summary)

rhat_max = summary["r_hat"].max()
ess_min = summary["ess_bulk"].min()
print(f"R_hat max: {rhat_max:.4f}")
print(f"ESS min: {ess_min:.0f}")

## 4 — Metricas: MAE, RMSE y R2 Poisson

In [ ]:
post = trace.posterior
alpha_m = float(post["alpha"].mean())
beta_d_m = float(post["beta_d"].mean())
beta_r_m = float(post["beta_r"].mean())
beta_w_m = float(post["beta_w"].mean())

def predict_lambda(d, r, w):
    return np.exp(alpha_m + beta_d_m * d + beta_r_m * r + beta_w_m * w)

lam_train = predict_lambda(d_train, r_train, w_train)
lam_val = predict_lambda(d_val, r_val, w_val)

mae_train = mean_absolute_error(y_train_np, lam_train)
mae_val = mean_absolute_error(y_val_np, lam_val)
rmse_train = np.sqrt(mean_squared_error(y_train_np, lam_train))
rmse_val = np.sqrt(mean_squared_error(y_val_np, lam_val))

def poisson_deviance(y, lam):
    y = np.array(y, dtype=float)
    lam = np.array(lam, dtype=float)
    safe = np.where(y > 0, y * np.log(y / lam) - (y - lam), -(y - lam))
    return 2 * safe.sum()

lam_null_train = np.full_like(lam_train, y_train_np.mean())
lam_null_val = np.full_like(lam_val, y_val_np.mean())

dev_model_train = poisson_deviance(y_train_np, lam_train)
dev_null_train = poisson_deviance(y_train_np, lam_null_train)
dev_model_val = poisson_deviance(y_val_np, lam_val)
dev_null_val = poisson_deviance(y_val_np, lam_null_val)

r2_train = 1 - dev_model_train / dev_null_train
r2_val = 1 - dev_model_val / dev_null_val

print(f"Train -> MAE {mae_train:.4f} | RMSE {rmse_train:.4f} | R2 {r2_train:.4f}")
print(f"Val   -> MAE {mae_val:.4f} | RMSE {rmse_val:.4f} | R2 {r2_val:.4f}")

## 5 — Overfitting, residuos y efectos

In [ ]:
delta_rel = (mae_val - mae_train) / mae_train * 100
status = "✅ Generaliza correctamente" if abs(delta_rel) < OVERFIT_THRESHOLD * 100 else "⚠️ Posible overfitting"
print(f"Delta relativo MAE: {delta_rel:+.2f}% -> {status}")

resid_train = (y_train_np - lam_train) / np.sqrt(lam_train)
resid_val = (y_val_np - lam_val) / np.sqrt(lam_val)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, resid, lam, label, color in zip(
    axes, [resid_train, resid_val], [lam_train, lam_val], ["Train", "Validacion"], [PALETTE["train"], PALETTE["val"]]
 ):
    ax.scatter(lam, resid, alpha=0.25, s=8, color=color, label=label)
    ax.axhline(0, color="black", lw=1.2, ls="--")
    ax.set_xlabel("lambda estimada")
    ax.set_ylabel("Residuo de Pearson")
    ax.set_title(f"Residuos - {label}")
    ax.legend()
plt.tight_layout()
plt.show()

delta_d_10 = 10 / discount_std
factor_d = np.exp(beta_d_m * delta_d_10)
delta_r_1 = 1 / rating_std
factor_r = np.exp(beta_r_m * delta_r_1)
factor_w = np.exp(beta_w_m)

print(f"+10 pp descuento -> x{factor_d:.4f} ({(factor_d-1)*100:+.2f}%)")
print(f"+1 punto rating  -> x{factor_r:.4f} ({(factor_r-1)*100:+.2f}%)")
print(f"Fin de semana    -> x{factor_w:.4f} ({(factor_w-1)*100:+.2f}%)")

## 6 — Persistencia de artefactos

In [ ]:
MODEL_DIR = Path("../models/modelo4")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

trace_path = MODEL_DIR / "modelo4_trace.nc"
trace.to_netcdf(str(trace_path))

scaler_data = {
    "discount_mean": discount_mean,
    "discount_std": discount_std,
    "rating_mean": rating_mean,
    "rating_std": rating_std,
    "metrics": {
        "mae_train": mae_train,
        "mae_val": mae_val,
        "rmse_train": rmse_train,
        "rmse_val": rmse_val,
        "r2_train": r2_train,
        "r2_val": r2_val,
        "delta_rel": delta_rel,
    },
}

scaler_path = MODEL_DIR / "modelo4_scaler.joblib"
joblib.dump(scaler_data, scaler_path)

print(f"✅ Trace guardado en: {trace_path}")
print(f"✅ Scaler/métricas guardado en: {scaler_path}")